In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg

# Third party import
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import imread
from tqdm import tqdm_notebook

# Package import
from modopt.math.metrics import ssim
import pysap
from pysap.data import get_sample_data
from mri.numerics.linear import WaveletUD
from pysap.numerics.fourier import FFT2
from pysap.numerics.fourier import NFFT
from pysap.numerics.gradient import GradAnalysis2
from modopt.opt.proximity import SparseThreshold
from mri.numerics.reconstruct import sparse_rec_condatvu
from pysap.numerics.utils import convert_mask_to_locations


                 .|'''|       /.\      '||'''|,
                 ||          // \\      ||   ||
'||''|, '||  ||` `|'''|,    //...\\     ||...|'
 ||  ||  `|..||   .   ||   //     \\    ||
 ||..|'      ||   |...|' .//       \\. .||
 ||       ,  |'
.||        ''

Package version: 0.0.3

License: CeCILL-B

Authors: 

Antoine Grigis <antoine.grigis@cea.fr>
Samuel Farrens <samuel.farrens@cea.fr>
Jean-Luc Starck <jl.stark@cea.fr>
Philippe Ciuciu <philippe.ciuciu@cea.fr>

Dependencies: 

scipy          : >=1.0.0   - required | 1.2.1     installed
numpy          : >=1.11.0  - required | 1.16.2    installed
matplotlib     : >=2.1.2   - required | 3.0.3     installed
future         : >=0.16.0  - required | 0.17.1    installed
astropy        : ==2.0.8   - required | 2.0.8     installed
nibabel        : >=2.1.0   - required | 2.4.0     installed
pyqtgraph      : >=0.10.0  - required | 0.10.0    installed
progressbar    : >=3.34.3  - required | 3.39.3    installed
modopt         : >=1.1.5   - requi

#  Loading input data

In [2]:
image = get_sample_data("mri-slice-nifti")
original_image_data = np.copy(image.data)
image.data += np.random.randn(*image.shape) * 20.
image.data = image.data.astype(np.complex128)
mask = get_sample_data("mri-mask")

In [3]:
non_cartesian = True
opt_results = {}

# Generate the kspace 
From the 2D brain slice and the acquistion mask, we generate the acquisition measurments, the observed kspace. We then reconstruct the zero order solution.

In [4]:
# Generate the subsampled kspace
if non_cartesian:
    kspace_loc = convert_mask_to_locations(mask.data)
    fourier_op = NFFT(samples=kspace_loc, shape=image.shape)
    kspace_data = fourier_op.op(image.data)

    # Zero order solution
    image_rec0 = pysap.Image(data=fourier_op.adj_op(kspace_data), metadata=image.metadata)
else:
    kspace_mask = np.fft.ifftshift(mask.data)
    kspace_loc = convert_mask_to_locations(kspace_mask)
    fourier_op = FFT2(
        samples=kspace_loc,
        shape=data.shape)
    kspace_data = fourier_op.op(image.data)
    
    # Zero order solution
    image_rec0 = pysap.Image(data=fourier_op.adj_op(kspace_data), metadata=image.metadata)

# Operators

In [5]:
nb_scales = 4
wavelet_id = 24

In [6]:
linear_op = WaveletUD(
    nb_scale=nb_scales,
    wavelet_id=wavelet_id,
    set_norm=1.1,
)

gradient_op = GradAnalysis2(
    data=kspace_data,
    fourier_op=fourier_op)

# Define the proximity dual/primal operator
prox_op = SparseThreshold(linear_op, None, thresh_type="soft")
cost_op = None

# Condat optimization
We now want to refine the zero order solution using a Condat optimization.

In [17]:
def objective_cost(x):
    return data_fidelity(x) + sparsity(x)

def sparsity(x):
    return mu * np.sum(np.abs(linear_op.op(x)))

def data_fidelity(x):
    return 0.5 * np.linalg.norm(fourier_op.op(x) - kspace_data)**2

In [18]:
def nrmse(x):
    return np.linalg.norm(x - original_image_data) / np.mean(original_image_data)

In [19]:
metrics_ = {
    "cost": {"metric": objective_cost, "mapping": {"x_new": "x"}, "cst_kwargs": {}, "early_stopping": False},
    "nrmse": {"metric": nrmse, "mapping": {"x_new": "x"}, "cst_kwargs": {}, "early_stopping": False},
}

In [20]:
mu = 80

In [21]:
max_iter = 50
convergence_pot = max_iter >= 500

In [22]:
# Start the FISTA reconstruction
cost_op = None
x_final, transform, costs, metrics = sparse_rec_condatvu(
    gradient_op,
    linear_op,
    prox_op,
    cost_op,
    std_est_method='dual',
    std_est=0.1,
    mu=mu,
    sigma=10,
    nb_of_reweights=0,
    max_nb_of_iter=max_iter,
    metrics=metrics_,
    metric_call_period=20,
    verbose=1,
)
image_rec = pysap.Image(data=np.abs(x_final))


   ____   U  ___ u  _   _    ____       _       _____      __     __    _   _
U /"___|   \/"_ \/ | \ |"|  |  _"\  U  /"\  u  |_ " _|     \ \   /"/uU |"|u| |
\| | u     | | | |<|  \| |>/| | | |  \/ _ \/     | |        \ \ / //  \| |\| |
 | |/__.-,_| |_| |U| |\  |uU| |_| |\ / ___ \    /| |\       /\ V /_,-. | |_| |
  \____|\_)-\___/  |_| \_|  |____/ u/_/   \_\  u |_|U      U  \_/-(_/ <<\___/
 _// \\      \\    ||   \\,-.|||_    \\    >>  _// \\_       //      (__) )(
(__)(__)    (__)   (_")  (_/(__)_)  (__)  (__)(__) (__)     (__)         (__)
    
 - mu:  80
 - lipschitz constant:  4.39999999999992
 - tau:  0.06993006988116797
 - sigma:  10
 - rho:  1.0
 - std:  0.1
 - 1/tau - sigma||L||^2 >= beta/2:  True
 - data:  (512, 512)
 - wavelet:  <mri.reconstruct.linear.WaveletUD object at 0x7f38fb9e7630> - 4
 - max iterations:  50
 - number of reweights:  0
 - primal variable shape:  (512, 512)
 - dual variable shape:  (10, 512, 512)
----------------------------------------
Starting optimiza

100% (50 of 50) |########################| Elapsed Time: 0:02:30 Time:  0:02:30


 - converged:  False
Done.
Execution time:  221.46993  seconds
----------------------------------------
